Importing Pandas and Numpy

In [1]:
import pandas as pd
import numpy as np

Printing 'Hello Capstone Project Course!'

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


Importing package for getting and parsing HTML file

In [38]:
from bs4 import BeautifulSoup
import requests
import lxml

Getting HTML file, source of a web-page

In [4]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

Making a 'soup', loading HTML file into BeautifulSoup parser

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

Finding necessary table in HTML file

In [6]:
table = soup.find('table')

Converting table into a Pandas DataFrame

In [11]:
df = pd.read_html(str(table))[0]

Lets watch at first and last 5 rows, using .tail() and .head() methods

In [12]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
df.tail()

,Postal Code,Borough,Neighborhood
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,NaN


As we can see, there are lots of 'Not assigned' ana 'Nan' cells. Lets deal with it

In [20]:
df.drop(df.loc[df.Borough == 'Not assigned', :].index, axis=0, inplace=True)

Lets watch at DataFrame again

In [29]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Reindexing rows

In [34]:
df.index = [i for i in range(len(df))]

Lets get shape of the DataFrame

In [37]:
df.shape

(103, 3)